In [18]:
import ete3
from Bio import SeqIO
import regex as re
import os
import pandas as pd

In [2]:
# read single fasta as infile :/
# extract uid from record.id and Tax= from record.description
# check if uid in a kog
# get taxid and write to uid_taxid file
# get domain and write to kog-level fasta file

In [3]:
#uid2kog["A6QBT0"]

In [20]:
uid2kog_df = pd.DataFrame(uid2kog.items())

In [23]:
uid2kog_df.rename(columns={0:"uid", 1:"kog"})
uid2kog_df.to_csv("/data/luojaa/uid2kogs.csv")

In [4]:
testpath = f"/data/luojaa/kegg/kog_uid/K00001.tsv"
base = "/data/luojaa/kegg/kog_uid/"
kog2uidpaths = [base + tsv for tsv in os.listdir(base)]
uid2kog = {} 
for path in kog2uidpaths:
    with open(path, "r") as f:
        for line in f:
            kog, uid, descriptor = line.strip().split("\t")
            uid2kog[uid] = kog

In [5]:
# def extract_taxa(instr):
#     taxa = re.findall(r"Tax=(.*)\sRepID", instr)[0]
#     return taxa
# def extract_uid(instr):
#     uid = re.findall(r"UniRef100_(.*)", instr)[0]
#     return uid
def extract_taxa(instr):
    taxa = re.findall(r"OX=([0-9]+)\s", instr)[0]
    return taxa
def extract_uid(instr):
    uid = instr.split("|")[1]
    return uid
#extract_taxa("tr|F0Q046|F0Q046_PARA1 Chromosomal replication initiator protein DnaA OS=Paracidovorax avenae (strain ATCC 19860 / DSM 7227 / CCUG 15838 / JCM 20985 / LMG 2117 / NCPPB 1011) OX=643561 GN=dnaA PE=3 SV=1")

In [6]:
def check_uid_coverage(infasta):
    uids =  set()
    with open(infasta, "r") as handle:
        for record in SeqIO.parse(handle, 'fasta'):
            identifier = record.id
            uid = extract_uid(identifier)
            uids.add(uid)
    return uids

In [5]:
#combined_set = check_uid_coverage("/data/luojaa/uniprot_combined.fasta")
#trembl_set = check_uid_coverage("/data/luojaa/uniprot_trembl.fasta")
#sprot_set = check_uid_coverage("/data/luojaa/uniprot_sprot.fasta")

In [6]:
#len(combined_set), len(trembl_set)

(248805733, 248234451)

In [104]:
#len(uid2kog)

12462195

In [14]:
# def taxa2taxid(taxa):
#     ncbi = ete3.NCBITaxa()
#     taxa2taxid = ncbi.get_name_translator([taxa])
#     taxid = taxa2taxid[taxa][0]
#     return taxid
ncbi = ete3.NCBITaxa()
def isEuk(taxid):
    lineage_ids = ncbi.get_lineage(taxid)
    if 2759 in lineage_ids:
        return True
    return False

In [26]:
uidset, taxidset = set(), set()
with open("/data/luojaa/uid2taxids.trembl.tsv", "r") as f:
    lines = f.readlines()
    for line in lines:
        uid, taxid = line.strip().split("\t")
        uidset.add(uid)
        taxidset.add(taxid)
len(uidset)

1932436

In [15]:
test_fasta = "/data/luojaa/uniprot_sprot.fasta"
#in_fasta = "/data/luojaa/uniprot_trembl.fasta"
uid_taxids = "/data/luojaa/uid2taxids.sprot.tsv"
log_path = "/data/luojaa/log/parse_fastas.log"
records_count, log_count, euk_count = 0, 0, 0
with open(test_fasta, "r") as handle, open(uid_taxids, "w") as outfile:
    for record in SeqIO.parse(handle, 'fasta'):
        if records_count % 100000 == 0:
            with open(log_path, "a") as logfile:
                print(",".join([str(records_count), "records_considered"]), file=logfile)
        records_count += 1
        identifier, description, sequence = record.id, record.description, record.seq
        uid = extract_uid(identifier)
        try:
            kog = uid2kog[uid]
            if log_count % 10000 == 0:
                with open(log_path, "a") as logfile:
                    print(",".join([str(log_count), "uids_mapped"]), file=logfile)
            log_count +=1
        except KeyError:
            continue
        taxid = extract_taxa(description)
        print("\t".join([uid, taxid]), file=outfile)
        if isEuk(taxid):
            if euk_count % 1000 == 0:
                with open(log_path, "a") as logfile:
                    print(",".join([str(euk_count), "euks_recorded"]), file=logfile)
            outfasta=f"/data/luojaa/kogfastas/{kog}.fasta"
            with open(outfasta, "a") as cluster:
                print(">" + uid, file=cluster)
                print(sequence, file=cluster)
            euk_count +=1
        
        

/data/luojaa/conda/envs/ete3/lib/python3.6/site-packages/ete3-3.1.3-py3.10.egg/ete3/ncbi_taxonomy/ncbiquery.py:243: UserWarning: taxid 103930 was translated into 47929
  warnings.warn("taxid %s was translated into %s" %(taxid, merged_conversion[taxid]))
